In [1]:
!pip install pandas dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2025-05-25 15:39:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-05-25 15:39:34 (889 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"

--2025-05-25 15:39:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2075 (2.0K) [application/x-python]
Saving to: ‘spacex-dash-app.py’

spacex-dash-app.py  100%[===================>]   2.03K  --.-KB/s    in 0s      

2025-05-25 15:39:44 (777 MB/s) - ‘spacex-dash-app.py’ saved [2075/2075]



In [17]:
!python spacex-dash-app.py

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'spacex-dash-app'
 * Debug mode: off
Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [22]:
# Ensure dash and its components are installed
# Ensure dash-core-components is installed
!pip install dash-core-components
# Install the JupyterDash helper
!pip install jupyter-dash

# Imports
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# 1) load your data
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# 2) instantiate the Dash app
app = dash.Dash(__name__)

# 3) define your layout and include the dropdown
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='site-dropdown',
        options=(
            [{'label': 'All Sites', 'value': 'ALL'}] +
            [{'label': site, 'value': site}
              for site in spacex_df['Launch Site'].unique()]
        ),
        value='ALL',
        placeholder='Select a Launch Site here',
        searchable=True
    ),

    html.Br(),
    dcc.Graph(id='success-pie-chart')
])

# 4) run the server
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

In [23]:
# Install the JupyterDash helper
!pip install jupyter-dash

# Imports
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# 1) Load the data
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# 2) Instantiate the Dash app
app = dash.Dash(__name__)

# 3) Define the layout
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center'}),

    # --- TASK 1: Launch Site dropdown ---
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site}
                 for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder='Select a Launch Site here',
        searchable=True
    ),

    html.Br(),

    # Pie chart placeholder
    dcc.Graph(id='success-pie-chart')
])

# 4) TASK 2: Callback to update the pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Filter only successful launches
        df = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(
            df,
            names='Launch Site',
            title='Total Success Launches By Site'
        )
    else:
        # Filter for the chosen site
        df = spacex_df[spacex_df['Launch Site'] == selected_site]
        # Count success vs. failure
        counts = (
            df['class']
              .map({1: 'Success', 0: 'Failure'})
              .value_counts()
              .reset_index()
        )
        counts.columns = ['Outcome', 'Count']
        fig = px.pie(
            counts,
            names='Outcome',
            values='Count',
            title=f'Launch Outcomes for {selected_site}'
        )

    return fig

# 5) Run the server (⚠️ this is the key change!)
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

In [27]:
# Install and import
!pip install jupyter-dash

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load data
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# Instantiate JupyterDash
app = JupyterDash(__name__)

# Layout
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign':'center'}),
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label':'All Sites','value':'ALL'}] +
                [{'label':s,'value':s} for s in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder='Select a Launch Site here',
        searchable=True
    ),
    html.Br(),
    dcc.Graph(id='success-pie-chart'),
    html.Hr(),
    html.P("Payload range (Kg):", style={'margin-top':'20px'}),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={i: str(i) for i in [0,2500,5000,7500,10000]},
        value=[min_payload, max_payload]
    ),
    html.Br(),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Pie chart callback
@app.callback(
    Output('success-pie-chart','figure'),
    Input('site-dropdown','value')
)
def update_pie(selected_site):
    if selected_site=='ALL':
        df = spacex_df[spacex_df['class']==1]
        return px.pie(df, names='Launch Site', title='Total Success Launches By Site')
    df = spacex_df[spacex_df['Launch Site']==selected_site]
    counts = (df['class'].map({1:'Success',0:'Failure'})
                .value_counts().reset_index())
    counts.columns=['Outcome','Count']
    return px.pie(counts, names='Outcome', values='Count',
                  title=f'Launch Outcomes for {selected_site}')

# Scatter callback
@app.callback(
    Output('success-payload-scatter-chart','figure'),
    [Input('site-dropdown','value'),
     Input('payload-slider','value')]
)
def update_scatter(site, payload_range):
    low, high = payload_range
    dff = spacex_df[(spacex_df['Payload Mass (kg)']>=low)&
                    (spacex_df['Payload Mass (kg)']<=high)]
    if site!='ALL':
        dff = dff[dff['Launch Site']==site]
    fig = px.scatter(
        dff, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Payload vs. Outcome'
    )
    fig.update_yaxes(title='Outcome (0 = Fail, 1 = Success)')
    return fig

# *** THIS IS THE KEY ***
# Run the server inline
app.run(mode='inline', debug=True)


/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

In [30]:
# Install and import
!pip install jupyter-dash
!pip install dash-core-components

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load data
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# Instantiate JupyterDash
app = JupyterDash(__name__)

# Layout
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign':'center'}),
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label':'All Sites','value':'ALL'}] +
                [{'label':s,'value':s} for s in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder='Select a Launch Site here',
        searchable=True
    ),
    html.Br(),
    dcc.Graph(id='success-pie-chart'),
    html.Hr(),
    html.P("Payload range (Kg):", style={'margin-top':'20px'}),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={i: str(i) for i in [0,2500,5000,7500,10000]},
        value=[min_payload, max_payload]
    ),
    html.Br(),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Pie chart callback
@app.callback(
    Output('success-pie-chart','figure'),
    Input('site-dropdown','value')
)
def update_pie(selected_site):
    if selected_site=='ALL':
        df = spacex_df[spacex_df['class']==1]
        return px.pie(df, names='Launch Site', title='Total Success Launches By Site')
    df = spacex_df[spacex_df['Launch Site']==selected_site]
    counts = (df['class'].map({1:'Success',0:'Failure'})
                .value_counts().reset_index())
    counts.columns=['Outcome','Count']
    return px.pie(counts, names='Outcome', values='Count',
                  title=f'Launch Outcomes for {selected_site}')

# Scatter callback - COMBINED HERE
@app.callback(
    Output('success-payload-scatter-chart','figure'),
    [Input('site-dropdown','value'),
     Input('payload-slider','value')]
)
def update_scatter(site, payload_range):
    low, high = payload_range
    dff = spacex_df[
        (spacex_df['Payload Mass (kg)']>=low)&
        (spacex_df['Payload Mass (kg)']<=high)
    ]
    if site!='ALL':
        dff = dff[dff['Launch Site']==site]
    fig = px.scatter(
        dff, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title=(
            'Correlation between Payload and Success '
            f"for {'All Sites' if site=='ALL' else site}"
        )
    )
    fig.update_yaxes(title='Launch Outcome (0 = Failure, 1 = Success)')
    return fig

# Run the server inline - ONLY RUN ONCE
app.run(mode='inline', debug=True)

/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

In [31]:
import pandas as pd

# 1) Load your data
df = pd.read_csv('spacex_launch_dash.csv')

# 2) Compute per‐site stats
site_success = df[df['class'] == 1].groupby('Launch Site').size().rename('Successes')
site_total   = df.groupby('Launch Site').size().rename('Total Launches')
site_stats   = pd.concat([site_total, site_success], axis=1).fillna(0)
site_stats['Success Rate'] = (site_stats['Successes'] / site_stats['Total Launches']).round(3)
site_stats = site_stats.sort_values('Successes', ascending=False)
print("=== Site Success Stats ===")
print(site_stats, "\n")

# 3) Compute payload‐range stats
bins   = [0, 2500, 5000, 7500, 10000]
labels = ['0–2500', '2500–5000', '5000–7500', '7500–10000']
df['Payload Range'] = pd.cut(df['Payload Mass (kg)'], bins=bins, labels=labels, include_lowest=True)
payload_stats = df.groupby('Payload Range')['class'].agg(
    Total_Launches='size',
    Successes=lambda x: (x==1).sum()
).reset_index()
payload_stats['Success Rate'] = (payload_stats['Successes'] / payload_stats['Total_Launches']).round(3)
print("=== Payload Range Success Stats ===")
print(payload_stats, "\n")

# 4) Compute booster‐version stats
booster_stats = df.groupby('Booster Version Category')['class'].agg(
    Total_Launches='size',
    Successes=lambda x: (x==1).sum()
).reset_index()
booster_stats['Success Rate'] = (booster_stats['Successes'] / booster_stats['Total_Launches']).round(3)
booster_stats = booster_stats.sort_values('Success Rate', ascending=False)
print("=== Booster Version Success Stats ===")
print(booster_stats)


=== Site Success Stats ===
              Total Launches  Successes  Success Rate
Launch Site                                          
KSC LC-39A                13         10         0.769
CCAFS LC-40               26          7         0.269
VAFB SLC-4E               10          4         0.400
CCAFS SLC-40               7          3         0.429 

=== Payload Range Success Stats ===
  Payload Range  Total_Launches  Successes  Success Rate
0        0–2500              21          7         0.333
1     2500–5000              21         12         0.571
2     5000–7500               9          2         0.222
3    7500–10000               5          3         0.600 

=== Booster Version Success Stats ===
  Booster Version Category  Total_Launches  Successes  Success Rate
1                       B5               1          1         1.000
2                       FT              24         16         0.667
0                       B4              11          6         0.545
4             

<ipython-input-31-84e068ef372e>:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

